<a href="https://colab.research.google.com/github/camilousa/SoftwareInteligente/blob/master/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate
import numpy as np

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/corredor-john/universidadsergioarboleda/main/datasetJC/TitanicData.csv")

In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 118 to 102
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  292 non-null    int64  
 1   Survived     292 non-null    int64  
 2   Pclass       292 non-null    int64  
 3   Name         292 non-null    object 
 4   Sex          292 non-null    object 
 5   Age          231 non-null    float64
 6   SibSp        292 non-null    int64  
 7   Parch        292 non-null    int64  
 8   Ticket       292 non-null    object 
 9   Fare         291 non-null    float64
 10  Cabin        69 non-null     object 
 11  Embarked     292 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 29.7+ KB


Preprocesamiento

In [ ]:
imputer1 = SimpleImputer(strategy='mean')

In [ ]:
imputer1.fit(train[['Age']])

SimpleImputer()

In [ ]:
imputer2 = SimpleImputer(strategy='median')
imputer2.fit(train[['Fare']])

SimpleImputer(strategy='median')

In [ ]:
train[['Age']] = imputer1.transform(train[['Age']])
# opción 2 imputer.transform(train[['Age']], copy=False)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 118 to 102
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  292 non-null    int64  
 1   Survived     292 non-null    int64  
 2   Pclass       292 non-null    int64  
 3   Name         292 non-null    object 
 4   Sex          292 non-null    object 
 5   Age          292 non-null    float64
 6   SibSp        292 non-null    int64  
 7   Parch        292 non-null    int64  
 8   Ticket       292 non-null    object 
 9   Fare         291 non-null    float64
 10  Cabin        69 non-null     object 
 11  Embarked     292 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 29.7+ KB


In [ ]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']

In [ ]:
column_transformer = ColumnTransformer(
    [("imputación con promedio", SimpleImputer(strategy='mean'), ['Age', 'Fare']),
     ("imputación con moda", SimpleImputer(strategy='most_frequent'), ['SibSp', 
                                                                       'Parch', 'Pclass']),
     ("eliminar",'drop', ['PassengerId', 'Cabin', 'Ticket', 'Name']),
    ], remainder='passthrough'
)

In [ ]:
column_transformer2 = ColumnTransformer(
    [
     ("Escalado con minmax", MinMaxScaler(),[0, 1, 2, 3]),
     
    ], remainder='passthrough'
)

In [ ]:
column_transformer3 = ColumnTransformer(
    [
     ("Categóricos a numérico", OneHotEncoder(),[5, 6]),
     
    ], remainder='passthrough'
)

In [ ]:
pipeline = Pipeline([('Paso 1 Imputación', column_transformer), 
                     ('Paso 2 escalado', column_transformer2),
                     ('Paso 3 categorico', column_transformer3)
                     ])

In [ ]:
X_train_transformed = pipeline.fit_transform(X_train)

In [ ]:
pd.DataFrame(X_train_transformed, columns=['Sex_female', 'Sex_male', 
                                           'Embarked_C', 'Embarked_Q', 'Embarked_S',
                                           'Age', 'Fare', 'SibSp', 'Parch', 'Pclass'
                                           ])

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age,Fare,SibSp,Parch,Pclass
0,0.0,1.0,1.0,0.0,0.0,0.471389,0.146862,0.0,0.0,1
1,0.0,1.0,0.0,1.0,0.0,0.405765,0.015127,0.0,0.0,3
2,1.0,0.0,0.0,0.0,1.0,0.828201,0.432884,0.125,0.0,1
3,0.0,1.0,1.0,0.0,0.0,0.286375,0.014102,0.0,0.0,3
4,0.0,1.0,1.0,0.0,0.0,0.696049,0.055628,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
287,0.0,1.0,0.0,0.0,1.0,0.27316,0.015412,0.0,0.0,3
288,0.0,1.0,0.0,1.0,0.0,0.27316,0.015265,0.0,0.0,3
289,0.0,1.0,1.0,0.0,0.0,0.603542,0.146862,0.0,0.0,1
290,0.0,1.0,0.0,0.0,1.0,0.312806,0.02635,0.0,0.0,2


In [ ]:
X_train_transformed[0:3, :]

array([[0.0, 1.0, 1.0, 0.0, 0.0, 0.4713889255979913, 0.1468620176246054,
        0.0, 0.0, 1],
       [0.0, 1.0, 0.0, 1.0, 0.0, 0.40576506441446314,
        0.015126992566498259, 0.0, 0.0, 3],
       [1.0, 0.0, 0.0, 0.0, 1.0, 0.8282014008193472, 0.4328841690069588,
        0.125, 0.0, 1]], dtype=object)

In [ ]:
import joblib

In [ ]:
joblib.dump(pipeline, "pipeline.joblib")

['pipeline.joblib']